# Training an Iris classifier using PyTorch
In this notebook, we'll show how to train a classifier trained on the [iris data set](https://archive.ics.uci.edu/ml/datasets/iris) using PyTorch.

## Install Dependencies
First, we'll install our dependencies:

In [0]:
pip install torch==1.2.* scikit-learn==0.21.* boto3==1.*

## Load the data
We can use scikit-learn to load the Iris dataset:

In [0]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

## Train the model
We'll use PyTorch to define the neural net and train the model:

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.metrics import accuracy_score


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)
        return X


train_X = Variable(torch.Tensor(X_train).float())
test_X = Variable(torch.Tensor(X_test).float())
train_y = Variable(torch.Tensor(y_train).long())
test_y = Variable(torch.Tensor(y_test).long())

model = Net()

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1000):
    optimizer.zero_grad()
    out = model(train_X)
    loss = criterion(out, train_y)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("number of epoch {} loss {}".format(epoch, loss))

predict_out = model(test_X)
_, predict_y = torch.max(predict_out, 1)

print("prediction accuracy {}".format(accuracy_score(test_y.data, predict_y.data)))  # Accuracy should be > 90%

## Export the model
Now we can export the model in the ONNX format:

In [0]:
placeholder = torch.randn(1, 4)
torch.onnx.export(
    model, placeholder, "pytorch.onnx", input_names=["input"], output_names=["species"]
)

## Upload the model to AWS

Cortex loads models from AWS, so we need to upload the exported model.

Set these variables to configure your AWS credentials and model upload path:

In [0]:
AWS_ACCESS_KEY_ID = "" #@param {type:"string"}
AWS_SECRET_ACCESS_KEY = "" #@param {type:"string"}
S3_UPLOAD_PATH = "s3://my-bucket/iris-classifier/pytorch.onnx" #@param {type:"string"}

import sys
import re

if AWS_ACCESS_KEY_ID == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_ACCESS_KEY_ID"), file=sys.stderr)

elif AWS_SECRET_ACCESS_KEY == "":
    print("\033[91m{}\033[00m".format("ERROR: Please set AWS_SECRET_ACCESS_KEY"), file=sys.stderr)

else:
    try:
        bucket = re.search("s3://(.+?)/", S3_UPLOAD_PATH).group(1)
        key = re.search("s3://.+?/(.+)", S3_UPLOAD_PATH).group(1)
    except:
        print("\033[91m{}\033[00m".format("ERROR: Invalid s3 path (should be of the form s3://my-bucket/path/to/file)"), file=sys.stderr)

Upload the model to S3:

In [0]:
import boto3

s3 = boto3.client("s3", aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
print("Uploading {} ...".format(S3_UPLOAD_PATH), end = '')
s3.upload_file("pytorch.onnx", bucket, key)
print(" ✓")

<!-- CORTEX_VERSION_MINOR -->
That's it! See the [example on GitHub](https://github.com/cortexlabs/cortex/tree/master/examples/iris-classifier) for how to deploy the model as an API.